# Assign wrkID for First Responsers

In [1]:
import pandas as pd
import numpy as npy
import geopy
import geopandas as gpd

## Read Data

In [4]:
#Load Census Tract
tract = gpd.read_file('../A6_Re_CreatePop_Jun/Data/Census_Cleaned.shp')#.set_index('GEOID10')
tract.crs

{'init': 'epsg:4269'}

In [4]:
#WrkID and Location
work = pd.read_csv('../Model_P/1_Results/wrkplace_Jun_Richard.csv').iloc[:,1:]

In [5]:
#hospital
hos = pd.read_csv('O_Tables/Hos_ID_Pop.csv')#.iloc[:,2:]
hos.head()

,Hospital,geometry,hosID,Pop
0,Englewood Hospital & Medical Center,POINT (-73.968469 40.903997),34003015200hs0,2500
1,Hackensack University Medical Center,POINT (-74.055982 40.88406),34003023402hs0,5000
2,Holy Name Hospital,POINT (-74.010501 40.882953),34003054500hs0,3000
3,CarePoint Health - Christ Hospital,POINT (-74.049665 40.734307),34017001400hs0,2000
4,RWJ Barnabas Health - Jersey City Medical Center,POINT (-74.05233699999998 40.717997),34017006500hs0,9000


In [12]:
#Fire Station
fire = pd.read_csv('O_Tables/Fire_coor.csv')
fire.head()

,FID,Latitude,Longitude,coord
0,0,40.703466,-74.007538,"(40.703466,-74.007538)"
1,1,40.710072,-74.012523,"(40.710072,-74.012523)"
2,2,40.710048,-74.005245,"(40.710048,-74.005245)"
3,3,40.715463,-74.005938,"(40.715463,-74.005938)"
4,4,40.719759,-74.006678,"(40.719759,-74.00667800000001)"


In [7]:
#Police Statiopn
pol = pd.read_csv('O_Tables/P_table.csv')#.iloc[:,]
pol.head()

,Precinct,Address,Lat,Long
0,1st Precinct,"16 Ericsson Place New York, NY, 10013",40.720270,-74.007198
1,5th Precinct,"19 Elizabeth Street New York, NY, 10013",40.716187,-73.997465
2,6th Precinct,"233 West 10 Street New York, NY, 10014",40.734233,-74.005453
3,7th Precinct,"19 1/2 Pitt Street New York, NY, 10002",40.716362,-73.983934
4,9th Precinct,"321 East 5 Street New York, NY, 10003",40.726559,-73.987824


In [13]:
workpop = pd.read_csv('work_pop.csv').iloc[:,1:]
workpop.head()

,wrkID,pop,long,lat,geometry
0,34003001000w0,1,-74.121686,41.045676,POINT (-74.12168600020868 41.04567600014735)
1,34003001000w1,2,-74.125241,41.041460,POINT (-74.12524099957244 41.04145999997905)
2,34003001000w10,2,-74.124340,41.031408,POINT (-74.12434000029072 41.03140799957959)
3,34003001000w100,10,-74.126718,41.044800,POINT (-74.12671800023548 41.04480000021989)
4,34003001000w101,7,-74.131662,41.033328,POINT (-74.13166199980139 41.03332799998725)


In [9]:
#change the file from csv to shp
def To_Gpd(data):
    from shapely.geometry import Point
    from shapely import wkt
    #data['geometry'] = hos['geometry'].apply(wkt.loads)
    try:
        data['geometry'] = data.apply(lambda x: Point((float(x.Longitude), float(x.Latitude))), axis = 1)
    except:
        data['geometry'] = data.apply(lambda x: Point((float(x.Long), float(x.Lat))), axis = 1)
    else:
        data['geometry'] = data['geometry'].apply(wkt.loads)
    data = gpd.GeoDataFrame(data, geometry='geometry')
    data.crs = {'epsg:4269'}
    return data

#### Police 

In [10]:
ps = To_Gpd(pol)
ps.crs

{'epsg:4269'}

#### Fire

In [ ]:
fire = To_Gpd(fire).iloc[:,[0,-1]]
fire.crs

#### Work Pop

In [ ]:
workpop = To_Gpd(workpop)

## Assign Unique HosID FireID and PSID

In [20]:
#Spatial Join
def join_s(fr,census):
    #hos
    gdf = gpd.sjoin(fr, census, how = "inner", op = 'intersects')
    #gdf = gdf.loc[:,['FID','geometry','GEOID10']]
    gdf = gdf.loc[:,['geometry','GEOID10']]
    gdf.rename(columns={'GEOID10': 'Census'}, inplace=True)
    #gdf['HosID'] = gdf.apply(lambda x:'%sh' % (x['Census']),axis=1)
    #gdf['FRID'] = gdf.apply(lambda x:'%sf' % (x['Census']),axis=1)
    gdf['ID'] = gdf.apply(lambda x:'%sp' % (x['Census']),axis=1)
    return gdf

In [89]:
#Calculate IDs
def assign_ID(data):
    cg = data.groupby(data['ID'])
    wp = pd.DataFrame()
    
    for i,j in cg:
        group = pd.DataFrame(j).reset_index(drop=True)
        for k in range (0,len(group)):
            group.iloc[k,2] = k
        wp = wp.append(group,ignore_index = True)
    
    wp['ID'] = wp.apply(lambda x:'%sp%s' % (x['Census'],x['ID']),axis=1)
    return wp

#### Hospital IDs

In [ ]:
hodid = join_sch(hos,tract)

In [526]:
hid = assign_schID(hodid).drop('Census',1)

#### FireID

In [ ]:
frid = join_s(fire,tract)

In [39]:
fid = assign_ID(frid).drop(['Census', 'FID'],1)

#### PoliceStation ID

In [84]:
psid = join_s(ps,tract)

In [92]:
pid = assign_ID(psid).drop(['Census'],1)

## Calculate the Distance to Assign the work places ID

In [94]:
def Cal_Dist(x, work):
    hosid = []
    wrkid = []
    for i in x.index:
        dist = work.distance(x.loc[i,'geometry']).sort_values()#.reset_index(drop = True)
        
        wrkid.append(work.loc[dist.index,'wrkID'].reset_index(drop=True))
        #AppendID
        hosid.append(x.iloc[i,-1])
    df = pd.DataFrame(wrkid)
    df['hosID'] = hosid
    return df

#### Fire

In [42]:
frdis = Cal_Dist(fid, workpop)

In [46]:
frdis =frdis.set_index('hosID').reset_index()

In [47]:
frdis.head()

,hosID,0,1,2,3,4,5,6,7,8,...,517087,517088,517089,517090,517091,517092,517093,517094,517095,517096
0,36005004100f0,36005005100w51,36005005100w174,36005005100w154,36005005100w89,36005005100w85,36005005100w10,36005005100w139,36005005100w96,36005005100w159,...,36103201004w20,36103201004w82,36103201004w42,36103170202w12,36103201004w74,36103201004w13,36103201004w63,36103201004w54,36103201004w84,36103201004w27
1,36005004600f0,36005004600w13,36005004600w5,36005004600w0,36005004600w4,36005004200w101,36005004200w103,36005004200w118,36005004200w129,36005004200w151,...,36103201004w20,36103201004w82,36103201004w42,36103170202w12,36103201004w74,36103201004w13,36103201004w63,36103201004w54,36103201004w84,36103201004w27
2,36005006500f0,36005006500w767,36005006500w22,36005006500w224,36005006500w196,36005006500w191,36005006500w66,36005006500w28,36005006500w460,36005006500w214,...,36103201004w20,36103201004w82,36103201004w42,36103170202w12,36103201004w74,36103201004w13,36103201004w63,36103201004w54,36103201004w84,36103201004w27
3,36005006900f0,36005006900w8,36005006700w16,36005006700w18,36005006900w6,36005006700w10,36005006700w21,36005006700w17,36005006900w5,36005006700w13,...,36103201004w20,36103201004w82,36103201004w42,36103170202w12,36103201004w74,36103201004w13,36103201004w63,36103201004w54,36103201004w84,36103201004w27
4,36005007900f0,36005008300w42,36005008300w56,36005008300w19,36005008300w52,36005008300w25,36005008300w16,36005008300w49,36005008300w21,36005008300w41,...,36103201004w20,36103201004w82,36103201004w42,36103170202w12,36103201004w13,36103201004w74,36103201004w63,36103201004w54,36103201004w84,36103201004w27


#### Police distance

In [103]:
psdis = Cal_Dist(pid, workpop)

In [105]:
psdis = psdis.set_index('hosID').reset_index()

In [106]:
psdis.head()

,hosID,0,1,2,3,4,5,6,7,8,...,517087,517088,517089,517090,517091,517092,517093,517094,517095,517096
0,36005003900p0,36005003900w17,36005003900w30,36005003900w15,36005003900w22,36005003900w38,36005003900w41,36005003900w6,36005003900w19,36005003900w25,...,36103170202w25,36103201004w82,36103201004w42,36103170202w12,36103201004w74,36103201004w13,36103201004w63,36103201004w54,36103201004w84,36103201004w27
1,36005004600p0,36005004600w5,36005004600w0,36005004600w4,36005004600w13,36005004600w9,36005004200w151,36005004200w126,36005004200w129,36005004200w42,...,36103201004w20,36103201004w82,36103201004w42,36103170202w12,36103201004w74,36103201004w13,36103201004w63,36103201004w54,36103201004w84,36103201004w27
2,36005008900p0,36005008900w7,36005008900w10,36005008900w12,36005008900w13,36005008900w2,36005008900w6,36005008900w11,36005008900w24,36005009300w422,...,36103201004w20,36103201004w82,36103201004w42,36103170202w12,36103201004w13,36103201004w74,36103201004w63,36103201004w54,36103201004w84,36103201004w27
3,36005014100p0,36005014100w2,36005014100w15,36005007100w2,36005007100w15,36005007100w68,36005007100w35,36005007100w44,36005007100w71,36005007100w84,...,36103201004w20,36103201004w82,36103201004w42,36103170202w12,36103201004w13,36103201004w74,36103201004w63,36103201004w54,36103201004w84,36103201004w27
4,36005016500p0,36005016500w25,36005039500w11,36005016500w13,36005016500w40,36005023100w32,36005023100w28,36005023100w18,36005023100w22,36005039500w10,...,36103201004w20,36103170202w12,36103201004w82,36103201004w42,36103201004w74,36103201004w13,36103201004w63,36103201004w54,36103201004w84,36103201004w27


#### Hos distance

In [ ]:
hosdis = Cal_Dist(hid,wrk)

In [ ]:
hosdis = hosdis.set_index('hosID').reset_index()

In [ ]:
hosdis.head()

In [ ]:
hosdis.shape

### Get work for Hospitals

In [ ]:
#1,first response deistance; 2,work table; 3,first responder with pop
def Find_wrkID(dis, work, fr):
    #New DataFrame to hold the results
    wp = pd.DataFrame()
    w = work.set_index('wrkID')
    #for i in range (0,4):
    for i in hosdis.index:
        print("This is ", i)
        #workID for the i hospital
        a = dis.iloc[i,1:].tolist()
        #hosID
        b = dis.loc[i,'hosID']
        print("HosId is:",b)
        #hosID
        Population = fr.loc[i, 'Pop']
    
        #Parameter for next loop
        flag = 0
        wrkid = []
        pop = []
        hos = []
        for j in a:
            #Condition to stop
            while flag < Population:
                c = w.loc[j,'pop']
                flag = flag + c      
                pop.append(c)
                wrkid.append(j)
                hos.append(b)
                break           
        sub = pd.DataFrame({'wrkID': wrkid, 'Pop': pop, 'hosID':hos})
        wp = wp.append(sub)
    return wp

In [107]:
#1,first response deistance; 2,work table; 3,first responder with pop;
def Find_wrkID_Police(dis, work, fr):
    #New DataFrame to hold the results
    wp = pd.DataFrame()
    w = work.set_index('wrkID')
    #for i in range (0,4):
    for i in dis.index:
        print("This is ", i)
        #workID for the i hospital
        a = dis.iloc[i,1:].tolist()
        #hosID
        b = dis.loc[i,'hosID']
        print("HosId is:",b)
        #hosID
        #Population = fr.loc[i, 'Pop']
        #Population = int(pop)
    
        #Parameter for next loop
        flag = 0
        wrkid = []
        pop = []
        hos = []
        for j in a:
            #Condition to stop
            while flag < 500:
                c = w.loc[j,'pop']
                flag = flag + c      
                pop.append(c)
                wrkid.append(j)
                hos.append(b)
                break           
        sub = pd.DataFrame({'wrkID': wrkid, 'Pop': pop, 'hosID':hos})
        wp = wp.append(sub)
    return wp

In [ ]:
#Get Hospital WrkID
df = Find_wrkID(hosdis, wrk, hid)

In [ ]:
#Firestation WrkID
firedf = Find_wrkID(frdis, workpop, fid)

In [ ]:
#Police Station WrkID
psdf = Find_wrkID(psdis, workpop, pid)